# this jupyter is to get all companies i want from our original dataset

In [1]:
from pymongo import MongoClient
import pandas as pd
import json as js
import folium

# Calling from mongo

It is a problem to have many companies with multiple office locations. I (we) solved that using "unwind", and "match" to link the whole query

In [40]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2005}},
            {"funding_rounds.raised_amount":{"$gt":1000000}}, { "$or":[{"category_code":"web"},
            {"category_code":"software"},{"category_code":"games_video"},
            {"category_code":"hardware"},{"category_code":"mobile"},{"category_code":"music"},
            {"category_code":"photo_video"},{"category_code":"design"}]}] }}
#    {"$match":"}
    ]
results = list(coll.aggregate(pipeline))

i had problems of unknown nature. I had a beautiful way to extract long and latitude in a a list to implement in folium, but now that is gone and i had to make up a patch 

In [41]:
location_raw=[]
name=[]
lon=[]
lat=[]

for element in results:
    name.append(element["name"])
    for coord in element["offices"].items():
        
        #print(coord)
        
        if coord[0]=="longitude":
            lon.append(coord[1])
        elif coord[0]=="latitude":
            lat.append(coord[1])

In [42]:
# to check 
print(len(name),len(lat),len(lon))

783 783 783


# Creating my dataframe

In [43]:
l={"name":name, "longitude":lon, "latitude":lat}
data_locations=pd.DataFrame(data=l)

In [44]:
data_clean=data_locations.dropna()

# Creating lists to insert in folium with coordinates and names

In [45]:
office_lat=list(data_clean["latitude"])
%store office_lat
office_long=list(data_clean["longitude"])
%store office_long
office_name=list(data_clean["name"])
%store office_name

Stored 'office_lat' (list)
Stored 'office_long' (list)
Stored 'office_name' (list)


In [53]:
data_clean["latitude"].dtypes

dtype('float64')